In [14]:
# reorganizing 
# start with the stuff you need to do once

from Bio.PDB import PDBParser, CaPPBuilder
from Bio import SeqIO
from Bio.Seq import Seq
from glob import glob 
from subprocess import check_output
import screed
from re import sub

p = PDBParser()
ppb = CaPPBuilder()

ecoli_codon = { 'G':'GGC', 'A':'GCG', 'V':'GTG', 'F':'TTT', 'E':'GAA', 'D':'GAT', 'N':'AAC', 
                'H':'CAT', 'P':'CCG', 'Q':'CAG', 'W':'TGG', 'Y':'TAT', 'I':'ATT', 'M':'ATG', 
                'C':'TGC', 'K':'AAA', 'L':'CTG', 'R':'CGT', 'T':'ACC', 'S':'AGC' }

def reverse_complement( oligo ):
    complement = { 'a': 't', 't': 'a', 'c': 'g', 'g': 'c', 'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C' } 
    reverse = [ complement[ i ] for i in oligo ][::-1]
    return ''.join( reverse )

bb_nucleotide = '1oaa.fn'
bb_pdb = '1oaa.pdb'
outpath = 'out/'

dna = SeqIO.read( bb_nucleotide, 'fasta' )
wt_structure = p.get_structure( 'wt_structure', bb_pdb )
wt_seq = [ pp.get_sequence() for pp in ppb.build_peptides( wt_structure ) ][ 0 ]

def make( design_structure, bb_nucleotide ):
    
    # design metadata 
    design = p.get_structure( 'design', design_structure )
    design_seq = [ pp.get_sequence() for pp in ppb.build_peptides( design ) ][ 0 ]
    design_fa = outpath + design_structure.replace( 'pdb', 'fa' )
    with open( design_fa, 'w' ) as handle:
        handle.write( '>{}\n{}'.format( design_structure, design_seq ) )

    # align design and scaffold 
    cmd = [ 'tblastn -query {} -subject {} -outfmt "6 sseq qseq sstart send"'.format( design_fa, bb_nucleotide ) ]
    blast_out = check_output( cmd, shell=True )
    best_hit = blast_out.split( '\n' )[ 0 ]
    sseq, qseq, sstart, send = best_hit.split( '\t' )

    # print a diff from this to use as the mutant handle 
    diff = '+'.join([ '{}{}{}'.format( native, position, designed ) for position, ( native, designed ) in enumerate( zip( sseq, qseq ) ) if native != designed ])

    # scaffold metadata 
    wt = [ record.sequence[ int( sstart ) - 1 : int( send ) ] for record in screed.open( bb_nucleotide ) ][0]
    codons = [ wt[i:i+3] for i in range( 0, len( wt ), 3 ) ]

    # mutate the scaffold sequence 
    for position, ( native, designed ) in enumerate( zip( wt_seq, design_seq ) ):
        if native != designed:
            if Seq( codons[ position ] ).translate() == native: # sanity check
                codons[ position ] = ecoli_codon[ designed ]

    oligos = sub( r'([atcg]{15})[atcg]{0,}([atcg]{15})', r'\1,\2', ''.join( codons ) ).split( ',' )[1:-1]
    with open( 'transcriptic_csv.csv', 'a' ) as transcriptic_csv:
        for oligo in oligos:
            transcriptic_csv.write( '{0},{1},{1},25nm,standard\n'.format( diff, reverse_complement( oligo ) ) )

    return ( diff, ''.join( codons ) )
            
# then loop through the mutants 
with open( 'transcriptic_csv.csv', 'w' ) as transcriptic_csv:
    transcriptic_csv.write( 'mutant_label,oligo_label,sequence,scale,purification\n' )

mutants = []
for mutant in glob( 'des*pdb' ):
    mutants += [ make( mutant, bb_nucleotide ) ]

In [ ]:
# run on transcriptic 

In [9]:
# now you've got sequencing results 
#import pandas
#pandas.DataFrame( mutants )

for mutant in mutants:
    mutant_nuc = outpath + mutant[0].lower() + '.fn'
    with open( mutant_nuc, 'w' ) as fn:
        fn.write( '>{}\n{}'.format( mutant[0], mutant[1] ) )

    #for i in [ 1, 2, 3 ]:
        #aseq_exp = 'reads/fwd/' + mutant[ 0 ] + '_{}'.format( i ) + '*.seq'
        #bseq_exp = 'reads/rev/' + mutant[ 0 ] + '_{}'.format( i ) + '*.seq'
        #aseq = glob( aseq_exp )[0]
        #aseq = glob( bseq_exp )[0]

        #cmd = [ 'merger', '-asequence', aseq, '-bsequence', bseq, '-outfile',  ]
        #check_output( cmd )
    
        #blast = [ 'blastn', '-subject', mutant_nuc, '-query', '' ]